In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 📘 CityAI – Decisions sobre inversions en inmobles

Aquest notebook utilitza un escenari plausible de l'Ajuntament de Barcelona per aplicar l'algorisme de la motxilla en un problema real.

## 🧠 Objectius

Aprofundir en els algorismes vistos a teoria, i veure la seva aplicació en un cas real.

## 🧩 Context

L'Ajuntament de Barcelona disposa de 20 milions d'euros per invertir en projectes immobiliaris de millora urbana. Té una llista de possibles inversions.

|            Projecte         | Cost (M€) | Retorn economic (M€) | Impacte social (sobre 10) |
| ----------------------------|-----------|----------------------|------------------|
| Habitatge social al Raval           | 8  | 10 | 9|
| Reforma mercat de Sant Antoni       | 6  | 7  | 6|
| Centre cívic a Nou Barris           | 4  | 5  | 8|
| Rehabilitació edificis Eixample     | 10 | 12 | 5|
| Oficina d'atenció ciutadana a Sants | 3  | 3  | 7|
| Habitatge sostenible al Poblenou    | 7  | 9  | 8|

Considerarem que el valor de cada inversió és:

valor = 0.8 x retorn_economic + 0.2 x impacte_social


In [3]:
# Per facilitar les proves, es proporcionen les dades inicials aquí mateix  

inversions = [
    ("Habitatge social al Raval", 8, 10, 9), 
    ("Reforma mercat de Sant Antoni", 6, 7, 6),
    ("Centre cívic a Nou Barris", 4, 5, 8),
    ("Rehabilitació edificis Eixample", 10, 12, 5),
    ("Oficina d'atenció ciutadana a Sants", 3, 3, 7),
    ("Habitatge sostenible al Poblenou", 7, 9, 8)
]
pressupost = 20

## ✍️ Exercici 1: Implementa l'algorisme de la motxilla amb les dades del cas

Tenim un conjunt d'opcions (immobles) cadascuna amb un cost i un benefici esperat (econòmic, social o mixt). L'Ajuntament disposa d'un pressupost màxim i vol maximitzar el benefici total.

Podem fer el paral·lelisme amb la motxilla:
+ Pes => cost d'inversió
+ Valor => tendiment esperat
+ Capacitat màxima => Pressupost total

Adapta el codi de la motxilla amb el cas descrit i amb el valor ponderat descrit més amunt.

Quines són les inversions triades?

In [9]:
from itertools import combinations
from typing import Any


def motxilla_fb(nombre:int, capacitat:int, pes_cost:list[any]) -> tuple[int,list[int]]:
    """
    Tria la millor combinació d'objectes amb l'algoritme de la motxilla 

    Parametres:
    :param nombre: nombre d'items possibles
    :param capacitat: capacitat de la motxilla
    :param pes_cost:
        llista de tuples del tipus [(pes, cost), (pes, cost), ...]

    Retorn:
        Tupla on el primer element és el cost i el segon element és la
        combinació òptima d'objectes.
    
    Idea:
        definition of status: 
            f(i, j) = **Maximum value** for a backpack with capacity j, in i items
        update status: whether to add the pes_cost[i] to the backpack
            f(i, j) = 0                                                                  if i == 0 or j == 0
                    = f(i-1, j)                                                          if j < i.weight
                    = max(f(i-1, j), f(i, j - pes_cost[i].weight) + pes_cost[i].value)   if j >= i-th item.weight
    """
    if not pes_cost: raise ValueError("There are 0 elements in pes_cost!!")
    comb_past: list[set[int]] = [set() for i in range(capacitat + 1)]
    comb_curr: list[set[int]] = [set() for i in range(capacitat + 1)]
    past: list[int] = [0] * (capacitat + 1)
    curr: list[int] = [0] * (capacitat + 1)

    # the case of backpacks for the 0-th item is the same as list prev, so
    # the dp starts from backpacks for the 1st item: pes_cost[0]
    for i in range(nombre):
        weight: int = pes_cost[i][0]
        value: float = pes_cost[i][1]

        for j in range(capacitat + 1):  # 有必要在这里将 range 改成 1，capacity + 1 吗？
            if j < weight:
                curr[j] = past[j]
                comb_curr[j] = comb_past[j].copy()
            else:
                take_item: int = past[j - weight] + value
                leave_item: int = past[j]
                if take_item > leave_item:
                    curr[j] = take_item
                    comb_curr[j] = comb_past[j - weight].copy()
                    comb_curr[j].add(i)
                else:
                    curr[j] = leave_item
                    comb_curr[j] = comb_past[j].copy()
        curr, past = past, curr
        comb_curr, comb_past = comb_past, comb_curr

    return (past[-1], comb_past[-1])

In [5]:
motxilla_fb(6, 20, [
    (8, 0.8 * 10 + 0.2 * 9), (6, 0.8 * 7 + 0.2 * 6), (48, 0.8 * 5 + 0.24 * 8), 
    (10, 0.8 * 12 + 0.2 * 5), (3, 0.8 * 3 + 0.2 * 7), (7, 0.8 * 9 + 0.2 * 8)
])

(23.200000000000003, {3, 4, 5})

Quines han sigut les inversions triades?

## ✍️ Exercici 2 – Discussió i reflexió

Què passaria si es donés més pes a l'impacte social que al retorn econòmic? per exemple 0.3 per retorn econòmic i 0.7 per retorn social.

Retoca l'algorisme o la crida i observa els resultats

In [10]:
motxilla_fb(6, 20, [
    ])

ValueError: There are 0 elements in pes_cost!!

Quines són ara les inversions triades? compara-les amb les anteriors.

## ✍️ Exercici 3 – Crea funcions auxiliars que 

1. Permetin dir el percentatge assignat a cada concepte (retorn econòmic i impacte social) i facin la crida corresponent a l'algorisme de la motxilla. 
1. Crein un diccionari amb la posició a la llista i el nom del projecte i quan l'algorisme retorni el resultat treguin un llistat dels projectes seleccionats.

In [7]:
def percentatge(ret_eco:float, imp_soc:float, llista:list[any]) -> (list[any]):
    """
    Calcula la combinació de projectes òptima en cost i impacte socioeconòmic. 

    Aquesta funció calcula la millor combinació de projectes d'una llista de
    projectes tot utilitzant l'algoritme de la motxilla, permetent personalitzar
    el balanç entre el pes donat al seu impacte social i el pes donat al seu
    retorn econòmic.

    Parametres:
        :param ret_eco: percentatge del retorn econòmic
        :param imp_soc: percentatge de l'impacte social
        :param llista: llista de tuples del tipus 
            [(projecte, pes, retorn_economic, impacte_social), ...]
            exemple: ("Habitatge social al Raval", 8, 10, 9)
    
    Retorn
        Tupla on el primer element és el cost i el segon element és la
        combinació òptima d'objectes.
    """
    pes_cost: list[tuple[int]] = [(i[1], i[2] * ret_eco + i[3] * imp_soc) for i in llista]

    return motxilla_fb(len(llista), 20, pes_cost)

In [8]:
percentatge(0.8, 0.2, inversions)

(25.000000000000004, {1, 2, 4, 5})

In [15]:
def resultats(selec:list[int],inversions:list[Any]) -> list[str]:
    """
    Tradueix la combinació de projectes a la llista amb els noms dels projectes.

    Parametres:
        :param selec: 
            llista amb els projectes seleccionats (1) i no seleccionats (0)
        :param inversions: 
            llista de tuples del tipus 
            [(projecte, pes, retorn_economic, impacte_social), ...]

    Retorn:
        Llista amb els noms dels projectes que té la combinació introduïda.
    """

    seleccionats: list[str] = [inversions[i][0] for i in range(len(selec)) if selec[i]]
   
    return seleccionats

In [16]:
resultats([1, 0, 0, 1, 0, 1], inversions)

['Habitatge social al Raval',
 'Rehabilitació edificis Eixample',
 'Habitatge sostenible al Poblenou']